# Image Classification
#### Created by Keenan McConkey on 2019.08.30

*Image classifier for terrains using PyTorch*

In [1]:
%matplotlib inline

import os
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
import torch.nn.functional as F

notebook_dir = os.path.dirname(os.path.realpath('__file__')) # Path to this notebook

In [6]:
img_dir = os.path.join(notebook_dir, 'terrain_imgs') # Img dir, each subdir is a class

def load_train_test_split(data_dir, train_frac = 0.8):
    # Preprocessing transforms for train/test data
    train_transforms = transforms.Compose([transforms.Resize(224), transforms.ToTensor()])
    test_transforms = transforms.Compose([transforms.Resize(224), transforms.ToTensor()])
    
    # Load eachs subdir as a class
    train_data = datasets.ImageFolder(data_dir, transform = train_transforms)
    test_data = datasets.ImageFolder(data_dir, transform = test_transforms)
    
    n_train = len(train_data)
    split = int(train_frac * n_train)
    
    # Shuffle and split
    indices = list(range(n_train))
    np.random.shuffle(indices)
    train_indices, test_indices = indices[:split], indices[split:]
    
    from torch.utils.data.sampler import SubsetRandomSampler
    train_sampler = SubsetRandomSampler(train_indices)
    test_sampler = SubsetRandomSampler(test_indices)
    
    # Return as a DataLoader to allow us to iterate thru the dataset
    from torch.utils.data import DataLoader
    train_loader = DataLoader(train_data, sampler = train_sampler, batch_size = 64)
    test_loader = DataLoader(test_data, sampler = test_sampler, batch_size = 64)
    
    return train_loader, test_loader

In [9]:
train_loader, test_loader = load_train_test_split(img_dir, train_frac = 0.8)

print('Train classes: %s' % train_loader.dataset.classes)
print('Test classes: %s' % test_loader.dataset.classes)

Train classes: ['asphalt', 'grass', 'gravel', 'sidewalk']
Test classes: ['asphalt', 'grass', 'gravel', 'sidewalk']


In [11]:
# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
print('Device: %s' % device)

Device: cuda


In [12]:
# Use pretrained ResNet-50 CNN for the convolutional layer
model = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/caris/.torch/models/resnet50-19c8e357.pth
24.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

63.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

98.1%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limi

In [13]:
print('Model: %s' % model)

Model: ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1),

In [14]:
# Freeze the pretrained CNN
for param in model.parameters():
    param.requires_grad = False

# Create a new fully connected layer for the new classes
model.fc = nn.Sequential(nn.Linear(2048, 512), nn.ReLU(), nn.Dropout(0.2), 
                         nn.Linear(512, 4), nn.LogSoftmax(dim=1))

# Set loss function for evaluating performance
criterion = nn.NLLLoss()

# Select optimization algorithm
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

# Send to GPu
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [ ]:
# Training parameters
epochs = 5
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []

# Train in epochs
for epoch in epochs:
    for train_data, train_labels in train_loader:
        # Increment step counter
        steps += 1
        
        # Send data to GPU
        train_data, train_labels = train_data.to(device), train_labels.to(device)
        